# 🌲 Binary Search Tree Operations

**Phase 3 (Interview Preparation) - Data Structures Module 2**

**Prerequisites**: Understanding of tree traversals

Master BST operations, validation, and common interview problems!

---

## 📊 BST Structure & Properties

Binary Search Tree with key structural and operational properties.

In [ ]:
// BST Node with type safety
case class BSTNode(value: Int,
                   left: Option[BSTNode] = None,
                   right: Option[BSTNode] = None) {
  
  // BST validation helpers
  def isValidBST: Boolean = {
    def validate(node: BSTNode, min: Option[Int], max: Option[Int]): Boolean = {
      val validLeft = node.left.forall { leftNode =>
        leftNode.value < node.value &&
        min.forall(_ <= leftNode.value) &&
        validate(leftNode, min, Some(node.value))
      }
      
      val validRight = node.right.forall { rightNode =>
        rightNode.value > node.value &&
        max.forall(_ >= rightNode.value) &&
        validate(rightNode, Some(node.value), max)
      }
      
      validLeft && validRight
    }
    
    validate(this, None, None)
  }
  
  // BST operations
  def insert(newValue: Int): BSTNode = {
    if (newValue <= value) {
      val newLeft = left.map(_.insert(newValue)).orElse(Some(BSTNode(newValue)))
      copy(left = newLeft)
    } else {
      val newRight = right.map(_.insert(newValue)).orElse(Some(BSTNode(newValue)))
      copy(right = newRight)
    }
  }
  
  def contains(searchValue: Int): Boolean = {
    if (searchValue == value) true
    else if (searchValue < value) left.exists(_.contains(searchValue))
    else right.exists(_.contains(searchValue))
  }
  
  def findMin: Int = left.map(_.findMin).getOrElse(value)
  def findMax: Int = right.map(_.findMax).getOrElse(value)
}

// BST companion object with factory methods
object BST {
  def apply(values: Int*): Option[BSTNode] = {
    values.headOption.map { first =>
      values.tail.foldLeft(BSTNode(first)) { (bst, value) =>
        bst.insert(value)
      }
    }
  }

  def isValidBST(root: Option[BSTNode]): Boolean = {
    root.forall(_.isValidBST)
  }
}

// Test BST operations
val bst = BST(8, 3, 10, 1, 6, 14, 4, 7, 13).get
println(s"BST constructed from [8, 3, 10, 1, 6, 14, 4, 7, 13]")
println(s"Is valid BST: ${bst.isValidBST}")
println(s"Contains 6: ${bst.contains(6)}")
println(s"Contains 9: ${bst.contains(9)}")
println(s"Min value: ${bst.findMin}")
println(s"Max value: ${bst.findMax}")
println()

## 🔍 BST Search Operations

Efficient search, insert, and delete operations in BST.

In [ ]:
// BST Search with path tracking
def searchWithPath(root: Option[BSTNode], target: Int): (Boolean, List[Int]) = {
  def searchHelper(node: Option[BSTNode], currentPath: List[Int]): (Boolean, List[Int]) = node match {
    case None => (false, currentPath)
    case Some(n) =>
      val newPath = currentPath :+ n.value
      if (n.value == target) (true, newPath)
      else if (target < n.value) searchHelper(n.left, newPath)
      else searchHelper(n.right, newPath)
  }
  searchHelper(root, Nil)
}

// Closest value in BST (for imperfect matches)
def findClosest(root: Option[BSTNode], target: Int): Int = {
  var closest = root.map(_.value).getOrElse(Int.MinValue)
  var current = root
  
  while (current.isDefined) {
    val nodeVal = current.get.value
    if (math.abs(target - nodeVal) < math.abs(target - closest)) {
      closest = nodeVal
    }
    
    current = if (target < nodeVal) current.get.left else current.get.right
  }
  
  closest
}

// Test search operations
println("BST Search Operations:")
val (found8, path8) = searchWithPath(Some(bst), 8)
println(s"Search for 8: Found=$found8, Path=$path8")

val (found6, path6) = searchWithPath(Some(bst), 6)
println(s"Search for 6: Found=$found6, Path=$path6")

val (found9, path9) = searchWithPath(Some(bst), 9)
println(s"Search for 9: Found=$found9, Path=$path9")

println(s"Closest to 9: ${findClosest(Some(bst), 9)}")
println(s"Closest to 5: ${findClosest(Some(bst), 5)}")
println(s"Closest to 12: ${findClosest(Some(bst), 12)}")
println()

## ➕ BST Insertion & Deletion

Complex operations maintaining BST invariant.

In [ ]:
// BST Deletion (finding successor for complex cases)
def deleteFromBST(root: Option[BSTNode], value: Int): Option[BSTNode] = {
  def findMin(node: BSTNode): BSTNode = {
    node.left.map(findMin).getOrElse(node)
  }
  
  def deleteHelper(node: BSTNode): BSTNode = {
    if (value < node.value) {
      node.copy(left = node.left.map(deleteHelper))
    } else if (value > node.value) {
      node.copy(right = node.right.map(deleteHelper))
    } else {
      // Node found - delete it
      (node.left, node.right) match {
        case (None, None) =>
          // No children
          null.asInstanceOf[BSTNode] // Will be filtered out
        case (Some(left), None) =>
          // Only left child
          left
        case (None, Some(right)) =>
          // Only right child
          right
        case (Some(left), Some(right)) =>
          // Two children - find inorder successor
          val successor = findMin(right)
          node.copy(
            value = successor.value,
            right = deleteFromBST(Some(right), successor.value)
          )
      }
    }
  }
  
  root.flatMap { r =>
    val result = deleteHelper(r)
    if (result == null) None else Some(result)
  }
}

// Batch operations
def insertBatch(root: Option[BSTNode], values: List[Int]): Option[BSTNode] = {
  values.foldLeft(root) { (current, value) =>
    current.map(_.insert(value)).orElse(Some(BSTNode(value)))
  }
}

def deleteBatch(root: Option[BSTNode], values: List[Int]): Option[BSTNode] = {
  values.foldLeft(root)(deleteFromBST)
}

// Test insertion and deletion
println("BST Modification Operations:")

// Insert operations
val bst2 = bst.insert(12).insert(15).insert(2)
println(s"After inserting [12, 15, 2]: min=${bst2.findMin}, max=${bst2.findMax}")

// Delete operations 
val bst3 = deleteFromBST(Some(bst2), 6).get
println(s"After deleting 6: BST still valid=${bst3.isValidBST}")
println(s"Find 6 after deletion: ${bst3.contains(6)}")

// Batch operations
val bst4 = insertBatch(Some(bst3), List(9, 11, 16))
println(s"After batch insert [9,11,16]: height=?")

val bst5 = deleteBatch(bst4, List(1, 4, 7))
println(s"After batch delete [1,4,7]: valid=${bst5.map(_.isValidBST).getOrElse(false)}")
println()

## 🎯 BST Validation Problems

In [ ]:
// Kth smallest element in BST
def kthSmallest(root: Option[BSTNode], k: Int): Option[Int] = {
  var count = 0
  var result: Option[Int] = None
  
  def inorder(node: Option[BSTNode]): Unit = node match {
    case Some(n) =>
      inorder(n.left)
      count += 1
      if (count == k) {
        result = Some(n.value)
        return
      }
      inorder(n.right)
    case None =>
  }
  
  inorder(root)
  result
}

// Range sum in BST
def rangeSum(root: Option[BSTNode], low: Int, high: Int): Int = {
  def sumHelper(node: Option[BSTNode]): Int = node match {
    case Some(n) =>
      if (n.value > high) sumHelper(n.left)
      else if (n.value < low) sumHelper(n.right)
      else n.value + sumHelper(n.left) + sumHelper(n.right)
    case None => 0
  }
  sumHelper(root)
}

// Convert BST to sorted doubly linked list
case class DLLNode(var value: Int, var prev: DLLNode = null, var next: DLLNode = null)

def bstToDLL(root: Option[BSTNode]): DLLNode = {
  var head: DLLNode = null
  var prev: DLLNode = null
  
  def inorderToDLL(node: Option[BSTNode]): Unit = node match {
    case Some(n) =>
      inorderToDLL(n.left)
      
      val current = DLLNode(n.value)
      if (head == null) head = current
      
      current.prev = prev
      if (prev != null) prev.next = current
      prev = current
      
      inorderToDLL(n.right)
    case None =>
  }
  
  inorderToDLL(root)
  head
}

// Test BST problems
println("BST Interview Problems:")

println(s"Kth smallest (k=3): ${kthSmallest(Some(bst), 3)}")
println(s"Kth smallest (k=7): ${kthSmallest(Some(bst), 7)}")

println(s"Range sum (3-10): ${rangeSum(Some(bst), 3, 10)}")
println(s"Range sum (1-5): ${rangeSum(Some(bst), 1, 5)}")

// BST to DLL test
val dllHead = bstToDLL(Some(BST(4, 2, 6, 1, 3, 5, 7).get))
print("BST to DLL: ")
var current = dllHead
while (current != null) {
  print(s"${current.value} ")
  current = current.next
}
println()
println()

## 🧭 BST Iterator & Successor/Predecessor

In [ ]:
// BST Iterator class
class BSTIterator(_root: Option[BSTNode]) {
  import scala.collection.mutable.Stack
  
  private val stack = Stack[BSTNode]()
  private var current: Option[BSTNode] = _root
  
  // Pre-load leftmost path
  private def leftmostInorder(node: Option[BSTNode]): Unit = {
    var curr = node
    while (curr.isDefined) {
      stack.push(curr.get)
      curr = curr.get.left
    }
  }
  
  leftmostInorder(_root)
  
  def hasNext: Boolean = stack.nonEmpty
  
  def next(): Int = {
    val node = stack.pop()
    val result = node.value
    
    // Add right subtree if exists
    leftmostInorder(node.right)
    
    result
  }
}

// Inorder successor in BST
def inorderSuccessor(root: BSTNode, target: Int): Option[Int] = {
  var successor: Option[Int] = None
  var current = Some(root)
  
  while (current.isDefined) {
    val node = current.get
    
    if (target >= node.value) {
      current = node.right
    } else {
      successor = Some(node.value)
      current = node.left
    }
  }
  
  successor
}

// Inorder predecessor in BST
def inorderPredecessor(root: BSTNode, target: Int): Option[Int] = {
  var predecessor: Option[Int] = None
  var current = Some(root)
  
  while (current.isDefined) {
    val node = current.get
    
    if (target <= node.value) {
      current = node.left
    } else {
      predecessor = Some(node.value)
      current = node.right
    }
  }
  
  predecessor
}

// Test BST iterator and successor
println("BST Iterator & Navigation:")

// Test iterator
val iterator = new BSTIterator(Some(bst))
print("BST Iterator: ")
var count = 0
while (iterator.hasNext && count < 8) {
  print(s"${iterator.next()} ")
  count += 1
}
println("...")

// Test successor and predecessor
println(s"Successor of 6: ${inorderSuccessor(bst, 6)}")
println(s"Successor of 7: ${inorderSuccessor(bst, 7)}")
println(s"Predecessor of 6: ${inorderPredecessor(bst, 6)}")
println(s"Predecessor of 3: ${inorderPredecessor(bst, 3)}")
println()

## 🎯 Interview Tips: BST Problems

### **Time Complexity Analysis:**
- **Average**: Search/Insert/Delete: O(log n), All traversals: O(n)
- **Worst**: O(n) for skewed trees (becomes linked list)
- **Space**: O(h) for recursion, O(n) for results

### **Common Patterns:**
1. **Tree Modification**: Insert/delete while maintaining BST property
2. **Validation**: Check BST invariants and handle corrupted trees
3. **Traversal-based**: Use inorder for sorted properties
4. **Binary Search**: Leverage BST property for efficient operations
5. **Range Queries**: Use BST for ordered range operations

### **Edge Cases:**
- Empty tree
- Single node
- Complete BST
- Left/right skewed
- Values equal to root
- Negative numbers
- Duplicate values
- Minimum/maximum values

### **Optimization Questions:**
- How to handle duplicates?
- Self-balancing variants (AVL, Red-Black)?
- Can you do this iteratively?
- Memory constraints?
- Thread safety considerations?